In [81]:
# Import packages and modules
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

In [83]:
df  = pd.read_csv("../Data/dataset_dk3619_preprocessed_v1.csv")

In [84]:
df.shape

(3323747, 43)

In [85]:
print(df['DK3619Code'])
dkcode_list = list(set(df['DK3619Code'].to_list()))
dkcode_list.sort()
id2dkcode = enumerate(dkcode_list)
id2dkcode = dict(id2dkcode)

dkcode2id = {}
for i in id2dkcode.items():
    dkcode2id[i[1]] = i[0]


print(dict(dkcode2id))

### Mapping data to numberic
# df['DK3619Code'] = df['DK3619Code'].map(dkcode2id)

0                 A_A
1                C_CA
2                C_CC
3          C_CE_CF_CD
4                C_CG
              ...    
3323742          Q_QB
3323743           R_R
3323744           S_S
3323745    C_CE_CF_CD
3323746          M_MA
Name: DK3619Code, Length: 3323747, dtype: object
{'A_A': 0, 'C_CA': 1, 'C_CC': 2, 'C_CE_CF_CD': 3, 'C_CG': 4, 'C_CH': 5, 'C_CI': 6, 'C_CJ': 7, 'C_CK': 8, 'C_CL': 9, 'C_CM_CB': 10, 'D_D': 11, 'E_E': 12, 'F_F': 13, 'G_G': 14, 'H_H': 15, 'I_I': 16, 'J_JA': 17, 'J_JB_JC': 18, 'K_K': 19, 'L_L': 20, 'M_MA': 21, 'M_MB': 22, 'M_MC': 23, 'N_N': 24, 'O_O': 25, 'PR_PR': 26, 'P_P': 27, 'Q_QA': 28, 'Q_QB': 29, 'R_R': 30, 'S_S': 31}


In [86]:
### Season mapping 

seasons_mapping = {
    'Spring': 0,
    'Summer': 1,
    'Autumn': 2,
    'Winter': 3
}

df['season'] = df['season'].map(seasons_mapping)

In [87]:
# Export the dkcode_list to a CSV file
dkcode_df = pd.DataFrame(dkcode_list, columns=['DK3619Code'])
dkcode_df.to_csv('dkcode_list.csv', index=False)

### Choose Feauture will use for Model

In [88]:
columns = ['HourUTC', 'HourDK', 'hour', 'day_of_week', 'month', 'year', 'day_of_year', 'day_of_month', 'season', 'holiday', 'lag_24', 'Consumption_MWh']


In [89]:
df_corr = df[columns]
df_corr.head()

,HourUTC,HourDK,hour,day_of_week,month,year,day_of_year,day_of_month,season,holiday,lag_24,Consumption_MWh
0,2024-06-21T21:00:00,2024-06-21 23:00:00,23,4,6,2024,173,21,1,0,NaN,145.018660
1,2024-06-21T21:00:00,2024-06-21 23:00:00,23,4,6,2024,173,21,1,0,NaN,189.913841
2,2024-06-21T21:00:00,2024-06-21 23:00:00,23,4,6,2024,173,21,1,0,NaN,27.199219
3,2024-06-21T21:00:00,2024-06-21 23:00:00,23,4,6,2024,173,21,1,0,NaN,174.295128
4,2024-06-21T21:00:00,2024-06-21 23:00:00,23,4,6,2024,173,21,1,0,NaN,105.906493


In [90]:
# Convert holiday to binary indicator
df_corr['holiday'] = df_corr['holiday'].apply(lambda x: 0 if x == 0 else 1)

/var/folders/82/s1cgdl3s5hn10rsmff2gt35r0000gn/T/ipykernel_5765/3026962626.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['holiday'] = df_corr['holiday'].apply(lambda x: 0 if x == 0 else 1)


### Split data

In [91]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,GRU, Dense, Bidirectional
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import TimeDistributed, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

import os.path


In [92]:
# Create sequences
def create_sequences(df, seq_length):
    sequences = []
    labels = []
    for i in range(len(df) - seq_length):
        seq = df.iloc[i:i+seq_length].values
        label = df['Consumption_MWh'].iloc[i+seq_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [96]:
#all_dkcode_trainings
if not os.path.exists("../Checkpoints/"):
    os.makedirs("../Checkpoints/")

for i in dkcode_list:
    if os.path.exists("../Checkpoints/" + i + "_LSTMmodel.h5"):
        continue
    print('Training LSTM model with DKCode: ', i)
    df_filter = df[df['DK3619Code'] == i]
    df_corr = df_filter[columns]
    df_corr['lag_24'] = df_corr['lag_24'].fillna(0)

    df_corr['HourUTC'] = pd.to_datetime(df_corr['HourUTC'])
    df_corr['HourDK'] = pd.to_datetime(df_corr['HourDK'])

    df_corr = df_corr.drop(['HourUTC', 'HourDK'], axis=1)

    # Normalize the data
    scaler = MinMaxScaler()
    
    #scaled_df = scaler.fit_transform(df_corr[['hour', 'day_of_week', 'month', 'year', 'day_of_year', 'day_of_month', 'season', 'holiday', 'lag_24', 'Consumption_MWh']])

    #scaled_df = pd.DataFrame(scaled_df, columns=['hour', 'day_of_week', 'month', 'year', 'day_of_year', 'day_of_month', 'season', 'holiday', 'lag_24', 'Consumption_MWh'])

    scaled_df = scaler.fit_transform(df_corr[['hour', 'day_of_week', 'month', 'year', 'day_of_year', 'day_of_month', 'season', 'holiday', 'Consumption_MWh']])

    scaled_df = pd.DataFrame(scaled_df, columns=['hour', 'day_of_week', 'month', 'year', 'day_of_year', 'day_of_month', 'season', 'holiday', 'Consumption_MWh'])
    
    #create sequence for training
    SEQ_LENGTH = 24  # for 24 hours sequence
    sequences, labels = create_sequences(scaled_df, SEQ_LENGTH)

    # Split the data into training (60%), validation (20%), and test (20%) sets
    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, shuffle=False)

    #LSTM modeling
    #SEQ_LENGTH=24 and  X_train.shape[2] = 6
    model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(SEQ_LENGTH, X_train.shape[2])),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])

    model.compile(optimizer='adam', loss='mean_squared_error')

    # Callbacks for early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

    # Model Training
    history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

    model.save("../Checkpoints/" + i + "_LSTMmodel.h5")
    model = tf.keras.models.load_model("../Checkpoints/" + i + "_LSTMmodel.h5")


    # Model Evaluation
    predictions = model.predict(X_test)

    # Calculate evaluation metrics
    r2 = r2_score(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)

    print(f'R² Score: {r2}')
    print(f'Mean Absolute Error (MAE): {mae}')
    print(f'Mean Squared Error (MSE): {mse}')
    print(f'Root Mean Squared Error (RMSE): {rmse}')

Training LSTM model with DKCode:  A_A


/var/folders/82/s1cgdl3s5hn10rsmff2gt35r0000gn/T/ipykernel_5765/3424048985.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['lag_24'] = df_corr['lag_24'].fillna(0)
/var/folders/82/s1cgdl3s5hn10rsmff2gt35r0000gn/T/ipykernel_5765/3424048985.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourUTC'] = pd.to_datetime(df_corr['HourUTC'])
/var/folders/82/s1cgdl3s5hn10rsmff2gt35r0000gn/T/ipykernel_5765/3424048985.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

Epoch 1/10
4226/4226 [==============================] - 29s 7ms/step - loss: 0.0041 - val_loss: 0.0037 - lr: 0.0010
Epoch 2/10
4226/4226 [==============================] - 27s 6ms/step - loss: 9.3134e-04 - val_loss: 0.0016 - lr: 0.0010
Epoch 3/10
4226/4226 [==============================] - 27s 6ms/step - loss: 4.7211e-04 - val_loss: 0.0010 - lr: 0.0010
Epoch 4/10
4226/4226 [==============================] - 32s 7ms/step - loss: 3.7326e-04 - val_loss: 0.0011 - lr: 0.0010
Epoch 5/10
4226/4226 [==============================] - 30s 7ms/step - loss: 3.3834e-04 - val_loss: 2.8834e-04 - lr: 0.0010
Epoch 6/10
4226/4226 [==============================] - 30s 7ms/step - loss: 3.1259e-04 - val_loss: 6.1030e-04 - lr: 0.0010
Epoch 7/10
4226/4226 [==============================] - 30s 7ms/step - loss: 2.9754e-04 - val_loss: 4.1516e-04 - lr: 0.0010
Epoch 8/10
4226/4226 [==============================] - 30s 7ms/step - loss: 2.8841e-04 - val_loss: 9.7403e-04 - lr: 0.0010
Epoch 9/10
4226/4226 [======

/Users/hung/anaconda3/envs/Env3_8/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


661/661 [==============================] - 2s 2ms/step
R² Score: 0.7862740214383727
Mean Absolute Error (MAE): 0.021565856973272644
Mean Squared Error (MSE): 0.0007949731518635814
Root Mean Squared Error (RMSE): 0.02819526825308781
Training LSTM model with DKCode:  C_CG


...

/Users/hung/anaconda3/envs/Env3_8/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


661/661 [==============================] - 2s 2ms/step
R² Score: 0.9656581198408357
Mean Absolute Error (MAE): 0.013375795143041186
Mean Squared Error (MSE): 0.00028206572149232245
Root Mean Squared Error (RMSE): 0.01679481233870514
Training LSTM model with DKCode:  C_CH


/var/folders/82/s1cgdl3s5hn10rsmff2gt35r0000gn/T/ipykernel_5765/3424048985.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['lag_24'] = df_corr['lag_24'].fillna(0)
/var/folders/82/s1cgdl3s5hn10rsmff2gt35r0000gn/T/ipykernel_5765/3424048985.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourUTC'] = pd.to_datetime(df_corr['HourUTC'])
/var/folders/82/s1cgdl3s5hn10rsmff2gt35r0000gn/T/ipykernel_5765/3424048985.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

Epoch 1/10
4226/4226 [==============================] - 31s 7ms/step - loss: 0.0052 - val_loss: 0.0014 - lr: 0.0010
Epoch 2/10
1568/4226 [==========>...................] - ETA: 19s - loss: 0.0020

KeyboardInterrupt: 